# Bimbo Advanced Price Elasticity GBM Proposal

### Libraries and General Settings

In [13]:
# Libraries needed
import pandas as pd
import numpy as np
#### ML
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error
### Others
import warnings
# ignore all warnings
warnings.filterwarnings("ignore")

In [14]:
# Constants and Parameters

### MONOTONIC CONSTRAINTS ARRAY
# Positions [so_unit_price, so_lag1, so_lag2,'promo_flag','holiday_flag','week_year'  ]
CONSTRAINTS = [-1, -1, -1, 1, 1, 0]

### INPUT DATA
FILE_NAME = '../inputs/fake_db_input.csv'

### OUTPUT
OUTPUT_FILE = '../outputs/reporte_elasticidad_lags.csv'

### Random State
SEED = 42

### Functions

In [15]:
def calculate_long_run_elasticity(model, X_sample, price_cols, percentage=0.01):
    """
    (EN) Simulates a permanent price increase (current and lags) to
    extract long time elasticity from the blackbox.
    
    (ES)Simula un incremento permanente del precio (actual y rezagos) 
    para extraer la elasticidad de largo plazo de la caja negra.
    """
    base_pred = model.predict(X_sample)
    base_pred = np.where(base_pred <= 0, 1e-9, base_pred)
    
    # Simulates a permanent increase (1%)
    X_sim = X_sample.copy()
    for col in price_cols:
        X_sim[col] = X_sim[col] * (1 + percentage)
    
    sim_pred = model.predict(X_sim)
    pct_change_q = (sim_pred - base_pred) / base_pred
    return np.mean(pct_change_q) / percentage


def write_final_report(df_report=pd.DataFrame(), file_name='', format='csv'):
    """
    (EN) Write final report with the modelling results.
    
    (ES) Escribe el reporte final con los resultados del modelamiento.
    """
    status = True

    try:
        # Classifying the results by case
        df_report['Case'] = np.where(df_report['Mape_Test'] > 0.2, 'MAPE fuera de rango admitido',
                            np.where(df_report['Elast_Sellout'] > 0, 'Elasticidad SO positiva',
                            np.where(df_report['Elast_Sellin'] > 0, 'Elasticidad SI Positiva',
                            'OK'  )))

        if format == 'csv':
            df_report.to_csv(file_name, index=False)
        elif format == 'excel':
            df_report.to_excel(file_name, index=False)  
        else:
            status = False
    except:
        status = False
    
    return status

### Modelling

In [16]:
# 1. Loading Data
df = pd.read_csv(FILE_NAME)
df['week_year'] = df['anio_sem'].astype(str).str[-2:].astype(int)

results = []
skus = df['sku'].unique()

for sku in skus:
    df_sku = df[df['sku'] == sku].sort_values('anio_sem').reset_index(drop=True)
    
    # Data requirement incremented to support lags (t-2)
    ### 20 will allow us to train with at least ~14 weeks and test with ~4
    if len(df_sku) < 20:
        continue
    
    # --- FEATURE ENGINEERING: Lag creation (Lags) ---
    # Sell-in Lags for Pass-through
    df_sku['si_lag1'] = df_sku['si_unit_price'].shift(1)
    df_sku['si_lag2'] = df_sku['si_unit_price'].shift(2)
    
    # Sell-out Lags for Demand
    df_sku['so_lag1'] = df_sku['so_unit_price'].shift(1)
    df_sku['so_lag2'] = df_sku['so_unit_price'].shift(2)
    
    df_clean = df_sku.dropna().copy()
    
    # --- Stage 1: Pass-through (Distributed Lag Model) ---
    X_pt = df_clean[['si_unit_price', 'si_lag1', 'si_lag2']]
    y_pt = df_clean['so_unit_price']
    
    pt_model = LinearRegression().fit(X_pt, y_pt)
    # The total pass-through is the sum of the coefficients (Cummulative Effect)
    total_pass_through = pt_model.coef_.sum()
    
    # --- Stage 2: Demand Model with Lags (GBM) ---
    price_features = ['so_unit_price', 'so_lag1', 'so_lag2']
    other_features = ['promo_flag', 'holiday_flag', 'week_year']
    X_demand = df_clean[price_features + other_features]
    y_demand = df_clean['so_volume']
    
    # Temp partition 
    split = int(len(df_clean) * 0.8)
    X_train, X_test = X_demand.iloc[:split], X_demand.iloc[split:]
    y_train, y_test = y_demand.iloc[:split], y_demand.iloc[split:]
    
    model_gb = HistGradientBoostingRegressor( max_iter=50,
                                              learning_rate=0.05,
                                              max_depth=2,
                                              min_samples_leaf=5,
                                              monotonic_cst=CONSTRAINTS,
                                              random_state=SEED)
    model_gb.fit(X_train, y_train)
    
    # Performance Metrics 
    mape_train = mean_absolute_percentage_error(y_train + 1e-9, model_gb.predict(X_train))
    mape_test = mean_absolute_percentage_error(y_test + 1e-9, model_gb.predict(X_test))
    
    # --- Stage 3: Elasticity Extraction  ---
    # Sell-Out Elasticity (Long Run)
    elast_so = calculate_long_run_elasticity(model_gb, X_demand, price_features)
    
    # Sell-In Elasticity (Effective impact to the company)
    elast_si = total_pass_through * elast_so
    
    results.append({
        'SKU': sku,
        'Pass_through': round(total_pass_through, 4),
        'Elast_Sellin': round(elast_si, 4),
        'Elast_Sellout': round(elast_so, 4),
        'Mape_Test': round(mape_test, 4),
        'Mape_Train': round(mape_train, 4)
    })

# Generating Final Output 
df_report = pd.DataFrame(results)
df_report.to_csv(OUTPUT_FILE, index=False)
print(df_report.head())

        SKU  Pass_through  Elast_Sellin  Elast_Sellout  Mape_Test  Mape_Train
0  SKU-1000        0.9490       -1.7422        -1.8358     0.1683      0.0736
1  SKU-1001        2.3088       -4.5711        -1.9799     0.1445      0.0843
2  SKU-1002        0.6238       -0.3067        -0.4917     0.2463      0.0626
3  SKU-1003        1.1744       -1.7807        -1.5163     0.0819      0.0389
4  SKU-1004        1.0644       -1.7317        -1.6269     0.1482      0.0455


In [17]:
# Saving Report File
write_final_report(df_report, OUTPUT_FILE)

True

In [18]:
# Checking Results
df_report.sort_values(by=['Case'],
                     ascending=False)

,SKU,Pass_through,Elast_Sellin,Elast_Sellout,Mape_Test,Mape_Train,Case
0,SKU-1000,0.9490,-1.7422,-1.8358,0.1683,0.0736,OK
1,SKU-1001,2.3088,-4.5711,-1.9799,0.1445,0.0843,OK
23,SKU-1023,0.9197,-2.8350,-3.0826,0.1347,0.0780,OK
22,SKU-1022,1.1522,-1.1762,-1.0208,0.0821,0.0466,OK
21,SKU-1021,1.7650,-4.4902,-2.5440,0.1031,0.0812,OK
19,SKU-1019,1.8460,-1.3843,-0.7499,0.1535,0.0683,OK
18,SKU-1018,1.3677,-0.8109,-0.5929,0.0901,0.0503,OK
17,SKU-1017,1.3353,-1.2341,-0.9242,0.1186,0.0726,OK
16,SKU-1016,1.1142,-1.2693,-1.1392,0.0965,0.0739,OK
15,SKU-1015,1.2036,-4.9921,-4.1476,0.1759,0.0765,OK
